## Move csv file from S3 to the head node and the to HDFS

In [1]:
%cd /mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Data/
!ls

/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Data
kmeans_data.txt  OldData  people.json  Weather


In [2]:
#create directory to hold data one node
!mkdir Weather
%cd Weather/

mkdir: cannot create directory ‘Weather’: File exists
/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Data/Weather


In [3]:
#list files on S3
!aws s3 ls s3://dse-weather/ALL.csv.gz
#compressed file is about 1.5GB

2016-02-09 04:09:49 1511481989 ALL.csv.gz


In [4]:
%%time
#copy file from S3
!aws s3 cp s3://dse-weather/ALL.csv.gz ./ALL.csv.gz

download: s3://dse-weather/ALL.csv.gz to ./ALL.csv.gz             
CPU times: user 2.02 s, sys: 960 ms, total: 2.98 s
Wall time: 25.8 s


In [5]:
%%time
#unompress file
!gunzip ALL.csv.gz

gzip: ALL.csv already exists; do you wish to overwrite (y or n)? ^C
CPU times: user 364 ms, sys: 164 ms, total: 528 ms
Wall time: 16.6 s


In [6]:
!ls -l ALL.csv
# About 7.7 GB

-rw-rw-r-- 1 hadoop hadoop 7668890105 Feb  9  2016 ALL.csv


In [7]:
!head -2 ALL.csv

station,year,measurement,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271

In [1]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
devtmpfs         7523076       28   7523048   1% /dev
tmpfs            7548784        0   7548784   0% /dev/shm
/dev/xvda1      10190136  3756720   6333148  38% /
/dev/xvdb1       5232640   110500   5122140   3% /emr
/dev/xvdb2      34052852 18053672  15999180  54% /mnt
/dev/xvdc       39294212    67924  39226288   1% /mnt1


In [2]:
#for i in range(10):
#    !cat ALL.csv >> ALL10.csv
#!ls -l ALL*

## Distribute file
copy file from the head-node file system to HDFS

In [ ]:
!hadoop fs -mkdir /weather

In [ ]:
%%time
#create a data directory on hdfs
!hadoop fs -copyFromLocal ALL.csv /weather/weather.csv

In [1]:
!hadoop fs -ls /weather

Found 3 items
drwxr-xr-x   - hadoop hadoop          0 2018-02-20 16:14 /weather/stations.parquet
-rw-r--r--   3 hadoop hadoop 7668890105 2018-03-01 17:17 /weather/weather.csv
drwxr-xr-x   - hadoop hadoop          0 2018-02-20 16:14 /weather/weather.parquet


### Read csv file into an RDD

In [3]:
!ls lib

binary_search.py   leaflet.py	  recon_plot2.py  YearPlotter.py
computeStats.py    MultiPlot.py   recon_plot.py
Eigen_decomp.py    numpy_pack.py  row_parser.py
import_modules.py  __pycache__	  spark_PCA.py


In [4]:
%%time
from pyspark import SparkContext
sc = SparkContext(pyFiles=['lib/numpy_pack.py'])

CPU times: user 248 ms, sys: 84 ms, total: 332 ms
Wall time: 21.4 s


In [5]:
%%time
RDD=sc.textFile('/weather/weather.csv')
print(RDD.count())

9358395
CPU times: user 28 ms, sys: 12 ms, total: 40 ms
Wall time: 26.2 s


In [6]:
fs_file="/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Data/Weather/ALL.csv"
!ls -l $fs_file

-rw-rw-r-- 1 hadoop hadoop 7668890105 Feb  9  2016 /mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Data/Weather/ALL.csv


In [7]:
%%time
with open(fs_file,'r') as f:
    text=f.readlines()
print(len(text))

9358395
CPU times: user 4.24 s, sys: 17.8 s, total: 22 s
Wall time: 24.8 s


In [ ]:
import os
os.env

In [ ]:
%%time
import sys
RDD=sc.textFile('hdfs://%s/weather/weather.csv'%sys.env['HOSTNAME'])

In [ ]:
%%time
RDD.take(2)

### Code for packing and unpacking byte arrays

In [ ]:
import numpy as np
"""Code for packing and unpacking a numpy array into a byte array.
   the array is flattened if it is not 1D.
   This is intended to be used as the interface for storing 
   
   This code is intended to be used to store numpy array as fields in a dataframe and then store the 
   dataframes in a parquet file.
"""

def packArray(a):
    """
    pack a numpy array into a bytearray that can be stored as a single 
    field in a spark DataFrame

    :param a: a numpy ndarray 
    :returns: a bytearray
    :rtype:

    """
    if type(a)!=np.ndarray:
        raise Exception("input to packArray should be numpy.ndarray. It is instead "+str(type(a)))
    return bytearray(a.tobytes())


def unpackArray(x,data_type=np.float16):
    """
    unpack a bytearray into a numpy.ndarray

    :param x: a bytearray
    :param data_type: The dtype of the array. This is important because if determines how many bytes go into each entry in the array.
    :returns: a numpy array
    :rtype: a numpy ndarray of dtype data_type.

    """
    return np.frombuffer(x,dtype=data_type)

In [ ]:
lines=RDD.take(10) # used to debug the code

In [ ]:
RDD.count()

### range values
Using code that was removed we find that the range of values is 

`-1000.0, 97892.0` 

which means that as ints we will need 32 but, but with float we can use just 16.

In [ ]:
#main parsing code

import numpy as np
def parse_weather(line):
    L=line.split(',')
    try:
        assert len(L)==368
        i=2
        L[i]=int(L[i])
        for i in range(3,368):
            if L[i]!='':
                L[i]=np.float16(L[i])
            else:
                L[i]=np.nan
    except:
        return None
        #print 'error in position',i, ':', line
    Out=L[:3]
    Out.append(packArray(np.array(L[3:],dtype=np.float16)))
    return Out

In [ ]:
lines=RDD.take(10)

In [ ]:
GG=parse_weather(lines[-2])

In [ ]:
DATA=RDD.map(parse_weather).filter(lambda x:x!=None) # fileter out bad rows which are mapped to None

In [ ]:
DATA.count()
#all lines: 9358395
# only the first line (the header) is bad.
# Good lines: 9358394

In [ ]:
DATA.take(1)

## Transform RDD into a Spark DataFrame

In [ ]:
import os
import sys

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType, BinaryType, FloatType

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
sqlContext

In [ ]:
### Defining the Schema explicitly
# The advantage of creating a DataFrame using a pre-defined schema allows the content of the RDD to be simple tuples, rather than rows.

# In this case we create the dataframe from an RDD of tuples (rather than Rows) and provide the schema explicitly
# Schema with two fields - person_name and person_age
schema = StructType([StructField("Station",     StringType(), True),
                     StructField("Measurement", StringType(), True),
                     StructField("Year",        IntegerType(),True),
                     StructField("Values",      BinaryType(),True)
                    ])
schema

In [ ]:
# Create a DataFrame by applying the schema to the RDD and print the schema
ALL_DataFrame = sqlContext.createDataFrame(DATA, schema)
ALL_DataFrame.printSchema()

### Write out data frame into Parquet directory

In [ ]:
%%time
!hadoop fs -rm -r /weather/weather.parquet

In [ ]:
%%time 
outfilename="hdfs://ec2-52-205-129-57.compute-1.amazonaws.com/weather/weather.parquet"
ALL_DataFrame.write.save(outfilename)

In [ ]:
!hadoop fs -ls /weather/

### Copy parquet directory to head node and then to S3

In [ ]:
%cd /mnt/workspace/Weather/
!rm -rf weather.parquet/
!ls -lrt

In [ ]:
%%time
!hadoop fs -copyToLocal /weather/weather.parquet weather.parquet

In [ ]:
!du *

In [ ]:
%%time
#copy file from S3
!aws s3 rm --recursive s3://dse-weather/weather.parquet
!aws s3 cp --recursive ./weather.parquet s3://dse-weather/weather.parquet